# Test the deployed web application

In this notebook, we test the local web app running inside the Docker container we made previously.

In [1]:
import numpy as np
import requests
import pandas as pd
import json

In [2]:
docker_login = 'fboylu'
image_name = docker_login + '/mlaksdep'

Run the Docker conatainer in the background and open port 80.

In [3]:
%%bash --bg -s "$image_name"
docker run -p 80:80 $1

Wait a few seconds for the application to spin up and then check that everything works.

In [5]:
!curl "http://0.0.0.0:80/"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (7) Failed to connect to 0.0.0.0 port 80: Address not available


In [9]:
!curl "http://0.0.0.0:80/version"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (7) Failed to connect to 0.0.0.0 port 80: Address not available


Now, let's use one of the duplicate questions to test our driver.

In [35]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

'count key/values in json.  possible duplicate: length of javascript associative array   i have a json that looks like this:  this prints when i do this:  i am trying to get the count of the items inside this json, obj.length returns "undefined" and obj[0].length returns   uncaught typeerror: cannot read property \'length\' of undefined  i would expect a length to return "2" in this case. how can i find the count? thanks!'

In [36]:
def text_to_json(text):
    return json.dumps({'input':'{0}'.format(text)})

In [37]:
jsontext = text_to_json(text_to_score)
jsontext[:100]

'{"input": "count key/values in json.  possible duplicate: length of javascript associative array   i'

In [38]:
headers = {'content-type': 'application/json'}
%time r = requests.post('http://0.0.0.0:80/score', data=jsontext, headers=headers)
print(r)
r.json()

CPU times: user 4.37 ms, sys: 139 µs, total: 4.51 ms
Wall time: 32.8 ms
<Response [500]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)